In [1]:
import pyodbc
import yfinance as yf
import pandas as pd

# Verbindungsstring
conn_str = (
    r'Driver=SQL Server;'
    r'Server=.\SQLEXPRESS;'
    r'Database=studienprojekt;'
    r'Trusted_Connection=yes;'
)

# Verbindung zur Datenbank herstellen
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

# Tabelle prüfen
table_name = "stock_data"  # Name der Zieltabelle
start_date = "2016-01-01"  # Startdatum, ab dem Daten überprüft werden sollen
end_date = "2023-07-07"  # Enddatum, bis zu dem Daten überprüft werden sollen

# Daten aus der Tabelle in einen DataFrame laden
select_query = f"SELECT * FROM {table_name}"
df_existing_data = pd.read_sql(select_query, cnxn)

# Ticker-Symbol definieren
ticker_symbol = "AAPL"

# Daten von yfinance abrufen
stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Datenverarbeitung
stock_data.reset_index(inplace=True)
stock_data.rename(columns={'Date': 'timestamp'}, inplace=True)
stock_data['timestamp'] = pd.to_datetime(stock_data['timestamp'])

# Erstellen Sie einen DataFrame zum Speichern von Indexwerten
df_index = pd.DataFrame(index=[], columns=["Index"])
df_index.index.name = "Company"

# Neue Funktion zum Verwalten von Indexwerten
def get_company_index(company, df_index):
    if company not in df_index.index:
        df_index.loc[company] = 1
    else:
        df_index.loc[company] += 1
    return df_index.loc[company]

# Daten nur einfügen, wenn sie noch nicht vorhanden sind
for _, row in stock_data.iterrows():
    timestamp = row['timestamp']
    existing_row = df_existing_data[(df_existing_data['timestamp'] == timestamp) & (df_existing_data['company'] == ticker_symbol)]
    if existing_row.empty:
        # Ermitteln Sie den Index für das Unternehmen
        company_index = get_company_index(ticker_symbol, df_index)
        # Daten in die Tabelle einfügen
        insert_query = f"INSERT INTO {table_name} (timestamp, [open], high, low, [close], volume, company, index) VALUES (?, ?, ?, ?, ?, ?, ?, ?)"
        cursor.execute(insert_query, timestamp, row['Open'], row['High'], row['Low'], row['Close'], row['Volume'], ticker_symbol, company_index)

# Änderungen in der Datenbank übernehmen
cnxn.commit()

print("Daten wurden erfolgreich in die SQL-Tabelle eingefügt.")


C:\Users\HerbyHerb\AppData\Local\Temp\ipykernel_17472\3454556005.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_existing_data = pd.read_sql(select_query, cnxn)


[*********************100%***********************]  1 of 1 completed


KeyError: 'company'